# Step 50, 미니배치를 뽑아주는 DataLoader

이전 단계에서는 Dataset 클래스를 만들어서 통일된 인터페이스로 데이터셋을 다룰 수 있게 하였다.  
이번 단계에서는 Dataset 클래스에서 미니배치를 뽑아주는 DataLoader 클래스를 구현한다.

DataLoader는 미니배치 생성, 데이터셋 shuffle 등의 기능을 제공하여 사용자가 작성해야 할 학습 코드가 더 간단해진다.

## 50.1 반복자

In [1]:
t = [1,2,3]
x = iter(t)
next(x)

1

In [2]:
next(x)

2

In [3]:
next(x)

3

In [4]:
next(x)

StopIteration: 

파이썬에서는 반복자를 직접 만들 수도 있다.  

기본 반복자를 만들어보고 반복자 구조를 이용하여 미니배치를 뽑아주는 DataLoader 클래스를 구현해 볼것이다.

In [5]:
# 반복자
class MyIterator:
    def __init__(self, max_cnt):
        self.max_cnt = max_cnt
        self.cnt = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.cnt == self.max_cnt:
            raise StopIteration()

        self.cnt += 1 
        return self.cnt 

클래스를 파이썬 반복자로 사용하려면  
\__iter\__ 특수 메서드를 구현, 자기 자신(self)을 반환  
\__next\__ 특수 메서드 다음 원소를 반환하도록 구현

In [6]:
obj = MyIterator(5)
for x in obj:
    print(x)

1
2
3
4
5


다음으로 반복자 구조를 이용하여 미니배치를 뽑아주는 DataLoader 클래스를 구현  
DataLoader는 주어진 데이터셋의 첫 데이터부터 차례로 꺼내주지만, 필요에 따라 shuffle 할 수도 있다.

In [7]:
# dezero/dataloaders.py 

import math 
import random
import numpy as np 

class DataLoader:
    def __init__(self, dataset, batch_size, shuffle=True):
        self.dataset = dataset          # Dataset 인터페이스를 만족하는 인스턴스
        self.batch_size = batch_size    # 배치 크기
        self.shuffle = shuffle          # 에포크별로 데이터셋을 shuffle 할지 여부
        self.data_size = len(dataset)
        self.max_iter = math.ceil(self.data_size / batch_size)

        self.reset()
    
    def reset(self):
        self.iteration = 0      # 반복 횟수 초기화 
        if self.shuffle:
            self.index = np.random.permutation(len(self.dataset))   # 데이터 shuffle
        else:
            self.index = np.arange(len(self.dataset))
    
    def __iter__(self):
        return self
    
    def __next__(self):         # 미니배치를 꺼내 ndarray 인스턴스로 변환
        if self.iteration >= self.max_iter:
            self.reset()
            raise StopIteration

        i = self.iteration
        batch_size = self.batch_size
        batch_index = self.index[i * batch_size : (i + 1) * batch_size]
        batch = [self.dataset[i] for i in batch_index]
        # 배치별로 나눔
        x = np.array([example[0] for example in batch])
        t = np.array([example[1] for example in batch])

        self.iteration += 1
        return x,t
    
    def next(self):
        return self.__next__()

마지막으로 dezero/\__init\__.py에 from dezero.dataloaders import DataLoader 추가 

## 50.2 DataLoader 사용하기 

In [8]:
from dezero.datasets import Spiral
from dezero import DataLoader 

batch_size = 10
max_epoch = 1

train_set = Spiral(train=True)
test_set = Spiral(train=False)

train_loader = DataLoader(train_set, batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size, shuffle=False)   

for epoch in range(max_epoch):
    for x, t in train_loader:
        print(x.shape, t.shape) # x,t는 train data
        break 

    # epoch 끝에서 test data를 꺼낸다.
    for x,t in test_loader:
        print(x.shape, t.shape) # x, t는 test data 
        break 

(10, 2) (10,)
(10, 2) (10,)


## 50.3 accuracy 함수 구현하기 

정확도를 평가해주는 accuracy 함수를 추가한다.

In [9]:
# dezero/functions.py 
from dezero import Variable, as_variable, as_array

def accuracy(y, t):
    y, t = as_variable(y), as_variable(t)

    pred = y.data.argmax(axis=1).reshape(t.shape)
    result = (pred == t.data)  # 텐서(ndarray)에 True/False 결과가 저장, 
    acc = result.mean()         # 이 텐서에 True 비율(평균)이 정답률
    return Variable(as_array(acc))

accuracy 함수는 인수 y와 t를 받아서 '정답률'을 계산  
y : 신경망의 예측 결과   
t : 정답 데이터

pred : 신경망의 예측 결과를의 최대 인덱스를 찾아서 형상을 변경하여 저장
pred와 정답 데이터 t를 비교하면 결과는 True/False의 텐서 (ndarray)가 된다.  
이 텐서의 True 비율(평균)이 정답률에 해당

## 50.4 스파이럴 데이터셋 학습 코드



In [10]:
import numpy as np 
import dezero 

from dezero import optimizers
from dezero import DataLoader
from dezero.models import MLP
import dezero.functions as F 

max_epoch = 300
batch_size = 30 
hidden_size = 10 
lr = 1.0

train_set = dezero.datasets.Spiral(train=True)
test_set = dezero.datasets.Spiral(train=False)

train_loader = DataLoader(train_set, batch_size)
test_loader = DataLoader(test_set, batch_size, shuffle=False)

model = MLP((hidden_size, 3))
optimizer = optimizers.SGD(lr).setup(model)

for epoch in range(max_epoch):
    sum_loss, sum_acc = 0, 0

    for x, t in train_loader:               # 1 훈련용 미니배치
        y = model(x)
        loss = F.softmax_cross_entropy(y,t)
        acc = F.accuracy(y,t)               # 2 훈련 데이터의 인식 정확도 
        model.cleargrads()
        loss.backward()
        optimizer.update()

        sum_loss += float(loss.data) * len(t)
        sum_acc += float(acc.data) * len(t)
    
    print('epoch: {}'.format(epoch+1))
    print('train loss: {:.4f}, accuracy: {:.4f}'.format(
        sum_loss / len(train_set), sum_acc / len(train_set)))
    
    sum_loss, sum_acc = 0, 0
    with dezero.no_grad():          # 3 기울기 불필요 모드 : 테스트 데이터셋에 모델을 테스트만 하는거니깐
        for x, t in test_loader:    # 4 테스트용 미니배치 데이터
            y = model(x)
            loss = F.softmax_cross_entropy(y, t)
            acc = F.accuracy(y, t)  # 5 테스트 데이터의 인식 정확도 
            sum_loss += float(loss.data) * len(t)
            sum_acc += float(acc.data) * len(t)

    print('test loss: {:.4f}, accuracy: {:.4f}'.format(
        sum_loss / len(test_set), sum_acc / len(test_set)))

: 0.6667
test loss: 0.6076, accuracy: 0.6967
epoch: 63
train loss: 0.5914, accuracy: 0.6700
test loss: 0.6020, accuracy: 0.6300
epoch: 64
train loss: 0.5911, accuracy: 0.6867
test loss: 0.5981, accuracy: 0.6533
epoch: 65
train loss: 0.5626, accuracy: 0.7000
test loss: 0.6124, accuracy: 0.7100
epoch: 66
train loss: 0.5707, accuracy: 0.7067
test loss: 0.5760, accuracy: 0.6567
epoch: 67
train loss: 0.5574, accuracy: 0.6900
test loss: 0.5834, accuracy: 0.6967
epoch: 68
train loss: 0.5560, accuracy: 0.7133
test loss: 0.5589, accuracy: 0.6933
epoch: 69
train loss: 0.5355, accuracy: 0.7267
test loss: 0.5530, accuracy: 0.6967
epoch: 70
train loss: 0.5303, accuracy: 0.7300
test loss: 0.5380, accuracy: 0.7267
epoch: 71
train loss: 0.5207, accuracy: 0.7167
test loss: 0.5315, accuracy: 0.7367
epoch: 72
train loss: 0.5094, accuracy: 0.7533
test loss: 0.5340, accuracy: 0.7367
epoch: 73
train loss: 0.5110, accuracy: 0.7400
test loss: 0.5127, accuracy: 0.7433
epoch: 74
train loss: 0.4921, accuracy: 0.

**NOTE_** 과대적합(overfitting)은 특정 훈련 데이터에 지나치게 최적화된 상태를 말한다.  
따라서 새로운 데이터에서는 예측 정확도가 훨씬 떨어지는, 일반화되지 못한 상태를 뜻한다.  
신경망으로는 표현력이 높은 모델을 말들 수 있기 때문에 과대적합이 흔히 일어난다.